In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'trexquant-2025-summer-intern-challenge-iitm:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F82591%2F9060719%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T202918Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db3c1b7a09d441fa3b039e41352e7723e5dd402b0b62a6dccddc7bc3c22a98c221cfe8d5ad6a6d5d823534a99e3c26cd0eb6d50a6431ecc11a87ed0a42c98306554cdc0923fde407992d36289e6469700440fb0bc15680ddb6f4b211843acf06c32f21865f274b090d68fa150a2a40c4032fc92b2583dae5ce411d66b8be3cffa243b3625ce394d8d2af49fc4bdf48e7b1098990d2349661b0d78cb2fcbb2c1cf05d70cdafc18169e53495ad33d635ddea709044a5f46af068da94551bc4e8d263cdf591ea2eedd1ff61e2a3abdf2f0b0c83256f1e180f2e423d54bf68d2f1645bced0d3078db8f2f0212dc8c9444f86335877febff6265f4882629a6595e513c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 68151584 bytes downloaded
Downloaded and uncompressed: trexquant-2025-summer-intern-challenge-iitm
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trexquant-2025-summer-intern-challenge-iitm/TQ_Train_Kaggle.csv
/kaggle/input/trexquant-2025-summer-intern-challenge-iitm/predictions_sandbox.csv
/kaggle/input/trexquant-2025-summer-intern-challenge-iitm/TQ_Test_Kaggle.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv('/kaggle/input/trexquant-2025-summer-intern-challenge-iitm/TQ_Train_Kaggle.csv')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
pd.set_option('display.max_rows', None)
df_train.isna().sum().sort_values(ascending=False)

In [ ]:
df_train.describe()

In [ ]:
df_train = df_train.apply(lambda x: x.fillna(0))
# df_train = df_train.apply(lambda x: x.fillna(x.mean()), axis=0)
# df_train = df_train.apply(lambda x: x.fillna(x.median()), axis=0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn import svm
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
!pip install catboost
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfrm = RandomForestRegressor(n_estimators=300, min_samples_split=5500, random_state=1)

train = df_train

predictors=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11','f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18','f19', 'f21', 'f22',
       'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32',
       'f33', 'f34', 'f35', 'f36', 'f37', 'f39', 'f40', 'f42', 'f43', 'f44',
       'f45', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f57',
       'f58', 'f59', 'f60', 'f61', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68',
       'f69', 'f70', 'f71', 'f72', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87']

rfrm.fit(train[predictors], train['target'])

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
xgbm = xgb.XGBRegressor(n_estimators=800,
                         learning_rate=0.001,
                         max_depth=6,
                         min_child_weight=2,
                         subsample=0.7,
                         colsample_bytree=0.9,
                         reg_alpha=0.001,
                         reg_lambda=1,
                         random_state=1  )



train = df_train
predictors=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11','f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18','f19', 'f21', 'f22',
       'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32',
       'f33', 'f34', 'f35', 'f36', 'f37', 'f39', 'f40', 'f42', 'f43', 'f44',
       'f45', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f57',
       'f58', 'f59', 'f60', 'f61', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68',
       'f69', 'f70', 'f71', 'f72', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87']
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7],
#     'min_child_weight': [1, 3, 5],
#     'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0]
# }

# grid_search = GridSearchCV(
#     estimator=initial_model,
#     param_grid=param_grid,
#     scoring='neg_mean_squared_error',
#     cv=5,
#     verbose=1,
#     n_jobs=-1
# )

# grid_search.fit(df_train[predictors], df_train['target'])

# best_model = grid_search.best_estimator_
xgbm.fit(train[predictors], train['target'])


In [ ]:
import lightgbm as lgb

lgbm = lgb.LGBMRegressor(
    n_estimators=800,
    learning_rate=0.005,
    max_depth=4,
    num_leaves=33,
    min_child_weight=2,
    subsample=0.7,
    colsample_bytree=0.9,
    reg_alpha=0.001,
    reg_lambda=1,
    random_state=1
)

train = df_train
predictors=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11','f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18','f19', 'f21', 'f22',
       'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32',
       'f33', 'f34', 'f35', 'f36', 'f37', 'f39', 'f40', 'f42', 'f43', 'f44',
       'f45', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f57',
       'f58', 'f59', 'f60', 'f61', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68',
       'f69', 'f70', 'f71', 'f72', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87']
lgbm.fit(train[predictors], train['target'])

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
knnm = KNeighborsRegressor(n_neighbors=10,
    weights='distance',
    algorithm='auto',
    leaf_size=30,
    metric='minkowski',
    p=2  )
# param_grid = {
#     'n_neighbors': [5, 10, 15, 20, 25],  # Try different k values
#     'weights': ['uniform', 'distance'],  # Different weighting strategies
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Different algorithms
#     'leaf_size': [10, 20, 30, 40, 50],  # Different leaf sizes
#     'metric': ['euclidean', 'manhattan', 'minkowski']  # Different distance metrics
# }

# random_search = RandomizedSearchCV(knn, param_distributions=param_grid, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=1, n_jobs=-1)
train = df_train
predictors=['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11','f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18','f19', 'f21', 'f22',
       'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32',
       'f33', 'f34', 'f35', 'f36', 'f37', 'f39', 'f40', 'f42', 'f43', 'f44',
       'f45', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f57',
       'f58', 'f59', 'f60', 'f61', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68',
       'f69', 'f70', 'f71', 'f72', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79',
       'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87']
knnm.fit(train[predictors], train['target'])

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, train[predictors], train['target'])

rmse_scores = (abs(cv_scores))**0.5

#replace {model} with the actual model

In [ ]:
df_test=pd.read_csv('TQ_Test_Kaggle.csv')
df_test.drop(columns=['f41','f38','f20','f73','f62','f80','f47','f46','f56'], inplace=True)
df_test =  df_test.apply(lambda x: x.fillna(0))
preds = model.predict(df_test[predictors])

In [ ]:
id=df_test['row_id']
submission_df = pd.DataFrame({
    'row_id': id,
    'Prediction': preds
})
submission_df.to_csv('submission.csv', index=False)